In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,dtype=tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,dtype=tf.int64)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [3]:
def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [4]:
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size = 100)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='elu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='elu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               160100    
_________________________________________________________________
dropout (Dropout)            (None, 100)              

In [11]:
NUM_EPOCHS = 10
        
early_stoping = tf.keras.callbacks.EarlyStopping()

model.fit(train_data, 
          epochs = NUM_EPOCHS,
          callbacks = [early_stoping],
          validation_data=(validation_inputs,validation_targets),
          verbose=2)

Epoch 1/10
540/540 - 14s - loss: 0.2105 - accuracy: 0.9361 - val_loss: 0.0769 - val_accuracy: 0.9748
Epoch 2/10
540/540 - 14s - loss: 0.0624 - accuracy: 0.9810 - val_loss: 0.0527 - val_accuracy: 0.9827
Epoch 3/10
540/540 - 14s - loss: 0.0461 - accuracy: 0.9849 - val_loss: 0.0353 - val_accuracy: 0.9897
Epoch 4/10
540/540 - 14s - loss: 0.0369 - accuracy: 0.9881 - val_loss: 0.0377 - val_accuracy: 0.9875


In [8]:
test_data, test_accuracy = model.evaluate(test_data)
print(test_data, test_accuracy)

1/1 [==============================] - 0s 1ms/step - loss: 0.0370 - accuracy: 0.9901
0.03704818710684776 0.9901000261306763
